In [ ]:
import DDQN_for_atari_space_invaders
import importlib
importlib.reload(DDQN_for_atari_space_invaders)

import numpy as np
import random
import time
import torch

In [2]:
import gymnasium
from gymnasium.wrappers import RecordVideo
import io
import base64
import glob
from IPython.display import HTML

In [7]:
def show_video():
    mp4_list = glob.glob('video/*.mp4')
    if len(mp4_list)==0:
        print('can not find vidoe')
        return
    
    video = mp4_list[0]
    video = io.open(video,'r+b').read()
    encoded = base64.b64encode(video)
    display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))


In [ ]:
env = DDQN_for_atari_space_invaders.make_env(render_mode='rgb_array')
env = RecordVideo(env,video_folder=f'video_{int(time.time())}',episode_trigger=lambda x:True,disable_logger=True)
policy_net = DDQN_for_atari_space_invaders.DQN(4,env.action_space.n)

# load our trained model
policy_net.load_state_dict(torch.load('policy_step_600000.pth',map_location='cpu'))
policy_net.eval()
device = 'mps'
policy_net.to(device)
random.seed(int(time.time()))
seed = random.randint(0,10000)
state,_ = env.reset(seed=seed)
state = torch.FloatTensor(state).unsqueeze(0).to(device)/255.0
count=0
while count<3:
    with torch.inference_mode():
        action = policy_net(state).max(1)[1].item()
    next_state,reward,terminated,truncated,_ = env.step(action)
    done = terminated or truncated
    if done:
        seed = random.randint(0,10000)
        state,_  = env.reset(seed=seed)
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        count+=1
        continue
    state = next_state
    state = torch.FloatTensor(state).unsqueeze(0).to(device)/255.0

env.close()
print('video recorded')

A.L.E: Arcade Learning Environment (version 0.11.2+ecc1138)
[Powered by Stella]


video recorded


In [ ]:
show_video()